In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tqdm import tqdm
from scipy.spatial.distance import cdist
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
timings = Path('timings/')
raw_data = Path('surface_data/raw/protein_surfaces/01-benchmark_surfaces_npy')

experiment_names = ['TangentConv_site_1layer_5A_epoch49', 'TangentConv_site_1layer_9A_epoch49', 'TangentConv_site_1layer_15A_epoch49','TangentConv_site_3layer_5A_epoch49','TangentConv_site_3layer_9A_epoch46', 'TangentConv_site_3layer_15A_epoch17','PointNet_site_1layer_5A_epoch30','PointNet_site_1layer_9A_epoch30','PointNet_site_3layer_5A_epoch46', 'PointNet_site_3layer_9A_epoch37', 'DGCNN_site_1layer_k40_epoch46','DGCNN_site_1layer_k100_epoch32','DGCNN_site_3layer_k40_epoch33']

experiment_names_short = ['Ours 1L 5A', 'Ours 1L 9A', 'Ours 1L 15A','Ours 3L 5A','Ours 3L 9A', 'Ours 3L 15A','PN++ 1L 5A','PN++ 1L 9A','PN++ 3L 5A', 'PN++ 3L 9A', 'DGCNN 1L K40','DGCNN 1L K100','DGCNN 3L K40']

In [3]:
performance = []
times = []
time_errors = []
memory = []
memory_errors = []
for experiment_name in experiment_names:

    predpoints_preds = np.load(timings/f'{experiment_name}_predpoints_preds.npy')
    predpoints_labels = np.load(timings/f'{experiment_name}_predpoints_labels.npy')

    rocauc = roc_auc_score(predpoints_labels,predpoints_preds)
    conv_times = np.load(timings/f'{experiment_name}_convtime.npy')
    memoryusage = np.load(timings/f'{experiment_name}_memoryusage.npy')
    memoryusage = memoryusage
    conv_times = conv_times

    performance.append(rocauc)
    times.append(conv_times.mean())
    time_errors.append(conv_times.std())
    memory.append(memoryusage.mean())
    memory_errors.append(memoryusage.std())


In [4]:
performance += [0.849]
times += [0.16402676922934395]
time_errors += [0.04377787154914341]
memory += [1491945956.9371428]
memory_errors += [125881554.73354617]
experiment_names_short += ['MaSIF 3L 9A']
colors += [40]

NameError: name 'colors' is not defined

In [151]:
experiment_names_short = [f'{i+1}) {experiment_names_short[i]}' for i in range(len(experiment_names_short))]

In [152]:
times = np.array(times)*1e3
time_errors = np.array(time_errors)*1e3
memory = np.array(memory)*1e-6
memory_errors = np.array(memory_errors)*1e-6

In [153]:
colors = [f'hsl(240,100,{25+i*10.83})' for i in range(6)]+[f'hsl(116,100,{25+i*16.25})' for i in range(4)] + [f'hsl(300,100,{25+i*21.66})' for i in range(3)] + [f'hsl(0,100,50)']

In [154]:
fig = go.Figure()
for i in range(len(times)):
    fig.add_trace(go.Scatter(
            x=[times[i]],
            y=[performance[i]],
            mode='markers',
            name=experiment_names_short[i],
            marker = dict(color=colors[i]),
            error_x=dict(
                type='data',
                symmetric=True,
                array=[time_errors[i]])))


fig.update_layout(
    xaxis_title='Forward pass time per protein [ms] (log)',
    yaxis_title='Site identification ROC-AUC',
    legend_title="Models",
)
fig.update_xaxes(type="log")
fig.update_layout(
    xaxis = dict(
        tickvals = [1e1,2e1,4e1,6e1,8e1,1e2,2e2,4e2,6e2],
        #tickvals = [10, 20, 50, 100, 200, 500],
    )
)

fig.show()
fig.write_image('figures/time_vs_perf.pdf')

In [155]:
fig = go.Figure()
for i in range(len(times)):
    fig.add_trace(go.Scatter(
            x=[memory[i]],
            y=[performance[i]],
            mode='markers',
            marker = dict(color=colors[i]),
            name=experiment_names_short[i],
            error_x=dict(
                type='data',
                symmetric=True,
                array=[memory_errors[i]])))


fig.update_layout(
    xaxis_title='Memory usage per protein [MB] (log)',
    yaxis_title='Site identification ROC-AUC',
    legend_title="Models",
)
fig.update_xaxes(type="log")
fig.update_layout(
    xaxis = dict(
        tickvals = [100,200,400,600,800,1000,2000,4000],
    )
)
fig.show()
fig.write_image('figures/mem_vs_perf.pdf')